In [1]:
import os
import numpy as np
import random
import cv2
import skimage.io
import matplotlib
import matplotlib.pyplot as plt

In [2]:
import mrcnn.config
from mrcnn import utils, visualize
from mrcnn.model import MaskRCNN
from pathlib import Path

C:\Users\dasun\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\dasun\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\dasun\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\dasun\anaconda3\envs\tensorflow\lib\site-packages

In [ ]:
#  Configuration that will be used by Mask R-CNN library

class MaskRCNNConfig(mrcnn.config.Config):
    NAME = "coco_pretrained_model_config"
    IMAGES_PER_GPU = 1
    GPU_COUNT = 1
    NUM_CLASSES = 1 + 80  # COCO dataset has 80 classes + one background class
    DETECTION_MIN_CONFIDENCE = 0.6
    
    
# Filter the result of Mask R-CNN to only obtain bounding boxes and class names of objects identified as cars

def getCarBoxes(boxes,class_ids):
    car_boxes = []
    
    # class_id 3/8 corresponds to car/truck objects as per COCO dataset
    for i,box in enumerate(boxes):
        if class_ids[i] in [3,8]:
            car_boxes.append(boxes)
            
    return np.array(car_boxes)

### <a id='section_2'>2. Initialize variables and paths</a>



In [ ]:
# Root directory of the project
ROOT_DIR = "."

# Local path for logs and trained model.One of the best practices and useful while training the model . 
# Will not be used in this demonstration
MODEL_DIR = os.path.join(ROOT_DIR,"logs")


# Path for saving trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR,"mask_rcnn_coco.h5")


# Downloading weights of pre-trained model for COCO dataset from the release 
# Executed for the first time when to store the model weights in this repo

# if not os.path.exists(COCO_MODEL_PATH):
#    utils.download_trained_weights(COCO_MODEL_PATH)
        

### <a id='section_3'>3. Creating model and loading model weights</a>


In [ ]:
# Directory of images to run detection on
IMAGE_DIR = "images"


# Create a Mask R-CNN model in inference mode
model = MaskRCNN(mode='inference', config=MaskRCNNConfig(),model_dir=MODEL_DIR)


# Load pre-trained model, this will load weights of model trained on COCO dataset
%time model.load_weights(COCO_MODEL_PATH,by_name=True)


For each object detected in the image fed as input, the model classifies it and returns class ID, which is an integer value identifying each class.

In [ ]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the person class, use: class_names.index('person')

class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

### <a id='section_4'>4. Identifying parking spaces in the parking lot</a>



The bouding box for each car identified is a parking parking spot.

These coordinates captured by the system in real time will be used for comparing bounding boxes of subsequent images captured in the system to find if the identified cars's bounding box overlap with the reference spots.

If the overlap is above a threshold, the space is occupied, otherwise it is **available** which is our aim.

In [ ]:
# Location of parking spaces
parked_car_boxes = None

### Run instance detection

In [ ]:
# Load a random image from images folder
file_names = next(os.walk(IMAGE_DIR))[2]         

In [ ]:
file_names

#### Identifying empty spots for overcast conditions

In [ ]:
# choose the first and 3rd image from a sequence of file_names 
# First image is the reference image of completely occupied parking area. It is used for finding all parking spots

image1 = skimage.io.imread(os.path.join(IMAGE_DIR,file_names[0]))
image2 = skimage.io.imread(os.path.join(IMAGE_DIR,file_names[2]))


In [ ]:
# run detection
# runs the detection pipeline and returns a list of dictionaries, one dict per image.

%time result1 = model.detect([image1])    


In [ ]:
%time result2 = model.detect([image2])    

In [ ]:
r1 = result1[0]
r2 = result2[0]


Visualize the result of instance segmentation 

In [ ]:
visualize.display_instances(image1,r1['rois'],r1['masks'], r1['class_ids'], class_names, r1['scores'])

In [ ]:
visualize.display_instances(image2,r2['rois'], r2['masks'], r2['class_ids'], class_names, r2['scores'])

In [ ]:
# Filter the results to only get identified cars' bounding boxes

car_boxes1 = getCarBoxes(r1['rois'],r1['class_ids'])
car_boxes2 = getCarBoxes(r2['rois'],r2['class_ids'])

# car_boxes1 and car_boxes2 is an array of length of no. of cars identified with each row having set of 
# 4 coordinates y1, x1, y2, x2 ; points 1 and 2 are opposite vertices of the bounding box.

In [ ]:
car_boxes1[0].shape

#### The first image has the parking lot occupied in full capacity. Hence, treating the bounding boxes for these cars as parking lot spaces in subsequent images to identify if these spaces are occupied by cars.

In [ ]:
parking_spaces = car_boxes1[0]

# computinig center locations of each spot
center_locs = []

for spot_coords in parking_spaces:
    center_locs.append([int((spot_coords[1]+spot_coords[3])/2), int((spot_coords[0]+spot_coords[2])/2)])


centers = np.array(center_locs)
    

In [ ]:
centers.shape

In [ ]:
# Draw each box on the frame

for i,box in enumerate(parking_spaces):
    y1, x1, y2, x2 = box
    cv2.rectangle(image1, (x1, y1), (x2, y2),(0,0,255),2)
    cv2.circle(image1,(centers[i][0],centers[i][1]),2,(0,0,255),2)

In [ ]:
# show the image on screen

cv2.imshow('image', image1)
cv2.waitKey(0)
cv2.destroyAllWindows()


The above snippet gives the following output for identifying all the parking spots

<img src='result_images/all_parking_spots.png'>

Bounding box and center location for an occupied spot is displayed in red. Since the location of a parking is the location of the parked car, bounding boxes and center location of all the cars are red.

In [ ]:
parking_spaces.shape

In [ ]:
car_boxes2.shape

### <a id='section_5'>5. Detecting empty parking spots</a>



In [ ]:
# How much car overlaps with the bounding boxes of parking spaces

overlaps = mrcnn.utils.compute_overlaps(car_boxes2[0],parking_spaces)

In [ ]:
overlaps.shape

In [ ]:
overlap_prob = overlaps.sum(axis=0)

In [ ]:
for i,box in enumerate(parking_spaces):
    y1, x1, y2, x2 = box
    
    if overlap_prob[i] >= 0.5:
        occupancy_status = (0,0,255)
    
    else:
        occupancy_status = (0,255,0)
    cv2.rectangle(image2,(x1,y1), (x2,y2) , occupancy_status ,1)
    cv2.circle(image2,(centers[i][0],centers[i][1]),2,occupancy_status,2)
        
    

In [ ]:
cv2.imshow('image', image2)
cv2.waitKey(0)
cv2.destroyAllWindows()

The above snippet give the following output for identifying empty parking spots in an image captured on a **overcast day**
    
<img src='result_images/detect_empty_space_overcast.png'>

Bounding box and center location for an **empty spot** is displayed in **green**.